In [24]:
from shared import getQdrantClient, getEmbeddingsModel
qClient = getQdrantClient()

# Show everything in the Document collection
numDocumentChunks = 0
# Note with_vectors defaults to false, so the vectors are not returned
chunks = qClient.scroll(collection_name='Document')
#print(chunks)
for chunk in chunks[0]:
    # Only display chunks if vector database is small
    print(chunk)
    if numDocumentChunks == 0:
        sampleDocumentChunk = chunk
    numDocumentChunks += 1
print("Number of document chunks: ", numDocumentChunks)
if numDocumentChunks > 0:
    print("\nSample document chunk(metadata not the vector): ")
    print(sampleDocumentChunk, '\n')

# Show everything in the Github collection
numGithubChunks = 0
chunks = qClient.scroll(collection_name='Github')
#print(chunks)
for chunk in chunks[0]:
    # Only display chunks if vector database is small
    print(chunk)
    if numGithubChunks == 0:
        sampleGithubChunk = chunk
    numGithubChunks += 1
print("\nNumber of Github chunks: ", numGithubChunks)
if numGithubChunks > 0:
    print("\nSample Github chunk(metadata not the vector): ")
    print(sampleGithubChunk, '\n')

# Show a sample search
embeddingsModel = getEmbeddingsModel()
results = qClient.search(
    collection_name="Document",
    query_vector = embeddingsModel.embed_query("What operating system is ROS made for?"),
    limit=2
)
print("\nSample search result(n=2): ")
for result in results:
    print(result)

id=0 payload={'link': 'https://www.ros.org/', 'type': 'Document', 'chunk': 0, 'text': ['ROS: Home Why ROS? Getting Started Community Ecosystem ROS - Robot Operating System The Robot Operating System (ROS) is a set of software libraries and tools that help you build robot applications. From drivers to state-of-the-art algorithms, and with powerful developer tools, ROS has what you need for your next robotics project. And it\'s all open source. What is ROS? ROS Videos " Install Jazzy Jalisco Jazzy Jalisco is our latest ROS 2 LTS release targeted at the Ubuntu 24.04 (Noble) and']} vector=None shard_key=None order_value=None
id=1 payload={'link': 'https://www.ros.org/', 'type': 'Document', 'chunk': 1, 'text': ['24.04 (Noble) and Windows 10, though other systems are supported to varying degrees. Learn More Humble Hawksbill ROS 2 Humble Hawksbill is a slighly older LTS release of ROS 2 targeted at Ubuntu 22.04 (Jammy) and Windows 10. Other systems are supported including tier 3 support for 2

In [12]:
import numpy as np
# How cosine distance works

embedding1 = embeddingsModel.embed_query("What is the weather like?")
embedding2 = embeddingsModel.embed_query("It is raining today.")
embedding3 = embeddingsModel.embed_query("ROS is an open source platform")
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)
similarity1 = cosine_similarity(embedding1, embedding2)
similarity2 = cosine_similarity(embedding1, embedding3)
print("Cosine Similarity for related sentences:", similarity1)
print("Cosine Similarity for unrelated sentences:", similarity2)

Cosine Similarity for related sentences: 0.7035977848391597
Cosine Similarity for unrelated sentences: 0.3566534327076298


In [20]:
from qdrant_client.http.models import Distance, VectorParams
# Delete all collections and vectors inside them
qClient.delete_collection(collection_name = "Document")
qClient.delete_collection(collection_name = "Github")
# Recreate the empty collections
qClient.create_collection(
    collection_name = "Document",
    vectors_config=VectorParams(size=3072, distance=Distance.COSINE)
)
qClient.create_collection(
    collection_name = "Github",
    vectors_config=VectorParams(size=3072, distance=Distance.COSINE)
)

True